In [ ]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
# https://github.com/PyTorchLightning/pytorch-lightning/issues/6210
!pip install neptune-client==0.5.1

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-0wx34cig
  Running command git clone -q https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-0wx34cig
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pytorch-lightning: filename=pytorch_lightning-1.3.0.dev0-cp37-none-any.whl size=832307 sha256=35f458fb9a75b3bfeb6ddc63b1cb012a12630a3ce2be42abf16ebf6a57d05397
  Stored in directory: /tmp/pip-ephem-wheel-cache-hsxjktsv/wheels/e2/c6/88/caa5d4cfbfab631fc84b0107896a6f661a1caf589160c27e71
Successfully built pytorch-lightning


In [ ]:
!pip install torch==1.7.0 torchvision==0.8 torchtext==0.8
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric
!pip install ogb

  Using cached https://files.pythonhosted.org/packages/d9/74/d52c014fbfb50aefc084d2bf5ffaa0a8456f69c586782b59f93ef45e2da9/torch-1.7.0-cp37-cp37m-manylinux1_x86_64.whl
  Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0


In [8]:
%load_ext autoreload
%autoreload 2
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
# from scikitplot.metrics import plot_confusion_matrix


from pytorch_lightning import Trainer
from lightning_model import LightningPAN
from pytorch_lightning.loggers.neptune import NeptuneLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Testing if the lightning_model script gets refreshed correctly
# LightningPAN.training_epoch_end("hello",'Hle')

HEL?


TypeError: ignored

In [11]:

# loading params
with open('parameters.json') as json_file:
    parameters = json.load(json_file)

datasetname = parameters["dataset_name"]
phi = parameters["phi"]
runs = parameters["runs"]
batch_size = parameters["batch_size"]
filter_size = parameters["maximum_path_size"]+1
learning_rate = parameters["learning_rate"]
weight_decay = parameters["weight_decay"]
pool_ratio = parameters["pool_ratio"]
nhid = parameters["nhid"]
epochs = parameters["epochs"]
LearningRateMonitor_Params = {'logging_interval': 'epoch'}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

model = LightningPAN(9, 1, nhid=nhid,ratio=pool_ratio,filter_size=filter_size)

lr_logger = LearningRateMonitor(**LearningRateMonitor_Params)
neptune_logger = NeptuneLogger(
                api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMDk5YjVmYzYtNTU0My00MzhkLWJiYTAtMGM4ZGVhZmEyMTZiIn0=",
                project_name='hvergnes/PAN',
                close_after_fit=False,
                params=parameters, # your hyperparameters, immutable
                tags=['PAN', 'fixed epoch end issues'],  # tags
                upload_source_files=["parameters.json", "lightning_model.py"]
                )

trainer = Trainer(
    max_epochs=epochs,
    # logger=neptune_logger,
    # callbacks=[lr_logger],
    gpus=1,
    # fast_dev_run=True,
)

model.cuda()
trainer.fit(model)
trainer.test(model)

NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name         | Type              | Params
----------------------------------------------------
0  | criterion    | BCEWithLogitsLoss | 0     
1  | atom_encoder | AtomEncoder       | 5.5 K 
2  | conv1        | PANConv           | 2.1 K 
3  | norm1        | Norm              | 192   
4  | pool1        | PANPooling        | 71    
5  | drop1        | PANDropout        | 0     
6  | conv2        | PANConv           | 4.2 K 
7  | norm2        | Norm              | 192   
8  | pool2        | PANPooling        | 69    
9  | drop2        | PANDropout        | 0     
10 | conv3        | PANConv           | 4.2 K 
11 | norm3        | Norm              | 192   
12 | pool3        | PANPooling        | 69    
13 | lin1         | Linear            | 2.1 K 
14 | bn1          | BatchNorm1d       | 64    
15 | lin2         | Linear            | 5

Device: cuda


Streaming output truncated to the last 5000 lines.
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[-0.08532078]
 [ 0.39785466]
 [ 0.07669356]
 [ 0.13029346]
 [-0.15382618]
 [-0.04694215]
 [ 0.6216314 ]
 [ 0.08270147]
 [ 0.28855434]
 [ 0.00870356]
 [ 0.2481238 ]
 [-0.18871713]
 [ 0.46443912]
 [ 0.6670607 ]
 [-0.23709539]
 [ 0.00499216]
 [ 0.40619603]
 [-0.01975938]
 [ 0.37940893]
 [-0.17574137]
 [ 0.15633322]
 [ 0.7700307 ]
 [ 0.3234116 ]
 [ 0.204656  ]
 [ 0.15624261]
 [ 0.31644845]
 [-0.3548886 ]
 [ 0.4302398 ]
 [ 0.12698343]
 [ 0.22900519]
 [ 0.06552023]
 [ 0.16023988]]
tensor(1.6488, device='cuda:0')
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]]
[[ 0.2569078 ]
 [ 0.09033252]
 [ 0.2489608 ]
 [ 0.12722835]
 [-0.43707097]
 [ 0.12065276]
 [ 0.1364139 ]
 [ 0.18026438]
 [ 0.01939584]
 [ 0.19826183]
 [-0.1786061 ]
 [ 0.03385736]
 [-0.6252

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:51: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


KeyboardInterrupt: ignored

In [ ]:
import os
print(os.listdir())
torch.save(model.state_dict(), 'model.pt')
print(os.listdir())